In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
X = data.data
y = data.target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import optuna.integration.lightgbm as lgb

# データセットの定義
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_val, label=y_val)

# パラメータの定義
params = {
    "objective": "regression",
    "metric": "l2",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

# Optunaの統合機能を使用してモデルのトレーニング
best_params, tuning_history = dict(), list()
model = lgb.train(
    params,
    dtrain,
    valid_sets=[dval],
    best_params=best_params,
    tuning_history=tuning_history,
    verbose_eval=100,
    early_stopping_rounds=100,
)

print("Best Params:", best_params)
print("Tuning history:", tuning_history)


In [ ]:
import optuna.integration.lightgbm as lgb

# データセットの定義
dtrain = lgb.Dataset(X, label=y)

# パラメータの定義
params = {
    "objective": "regression",
    "metric": "l2",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

# Optunaの統合機能を使用してクロスバリデーション
best_params, tuning_history = dict(), list()
result = lgb.cv(
    params,
    dtrain,
    nfold=5,
    stratified=False,
    best_params=best_params,
    tuning_history=tuning_history,
    verbose_eval=100,
    early_stopping_rounds=100,
    return_cvbooster=True
)

print("Best Params:", best_params)
print("Tuning history:", tuning_history)


In [ ]:
# 1. 最も良いスコアのラウンドを取得
best_round = result['l2-mean'].index(min(result['l2-mean']))

# 2. そのラウンドでのモデルを取得
best_models = result['cvbooster'].boosters

# 各フォールドでのモデルのスコアを取得し、最も良いスコアを持つモデルを選択
best_score = float('inf')
best_model = None
for model in best_models:
    score = model.best_score['valid_0']['l2']
    if score < best_score:
        best_score = score
        best_model = model

print("Best model's score:", best_score)


In [ ]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras

# データのロード
data = load_boston()
X = data.data
y = data.target


In [ ]:
def build_model(input_shape):
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=input_shape),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
models = []
histories = []
scores = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = build_model((X_train.shape[1],))
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=0)

    models.append(model)
    histories.append(history.history)

    y_pred = model.predict(X_val)
    score = mean_squared_error(y_val, y_pred)
    scores.append(score)

best_index = np.argmin(scores)
best_model = models[best_index]

print(f"Best Model's MSE: {scores[best_index]}")



In [ ]:
import numpy as np
import optuna
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras

# 1. データの準備
data = load_boston()
X = data.data
y = data.target
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

# 2. モデル定義
def build_model(input_shape, n_layers, n_units, dropout_rate):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for _ in range(n_layers):
        model.add(keras.layers.Dense(n_units, activation='relu'))
        model.add(keras.layers.Dropout(dropout_rate))
    model.add(keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# 3. Optunaの目的関数
def objective(trial):
    n_layers = trial.suggest_int('n_layers', 1, 5)
    n_units = trial.suggest_int('n_units', 32, 128, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    model = build_model((X_train.shape[1],), n_layers, n_units, dropout_rate)
    model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=0)
    y_pred = model.predict(X_test)
    score = mean_squared_error(y_test, y_pred)
    return score

# 4. Optunaでのハイパーパラメータの最適化
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# 5. 最適なハイパーパラメータの表示
best_params = study.best_params
print("Best parameters: ", best_params)
